## get tad clusters

put this into the pipline eventually??


In [1]:
import pandas as pd
import numpy as np
import os
from tqdm.auto import tqdm  # for notebooks

In [2]:
# load in the tissue ids 
prefix = '/home/klawren/oak/pcqtls'
import yaml
config_path= f'{prefix}/config/pcqtl_proteincoding.yaml'
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

tissue_id_path = config['tissue_id_path']
expression_dir = config['expression_dir']

tissue_df = pd.read_csv(f"{prefix}/{tissue_id_path}", header=0)
tissue_ids = list(tissue_df['Tissue'])

In [3]:
# load in the tad data
# data from http://dna.cs.miami.edu/TADKB/search.php
tad_path = '/home/klawren/oak/pcqtls/data/references/TAD_annotations/TADs_hg38/converted_HiC_IMR90_DI_10kb.txt'

tad_df = pd.read_csv(tad_path, header=None, sep='\t', names=['chr', 'start','end'])
tad_df['Chromosome'] = tad_df['chr'].str.strip('chr')
tad_df.head()

,chr,start,end,Chromosome
0,chr1,784620,1314620,1
1,chr1,1374620,1514620,1
2,chr1,1644639,1748561,1
3,chr1,1778561,1908561,1
4,chr1,1918561,2178561,1


In [4]:
# load in the gene data
# load in the gene information (start and strand are what I need)
full_gencode=pd.read_csv('/home/klawren/oak/pcqtls/data/references/processed_gencode.v26.GRCh38.genes.gtf', sep='\t', skiprows=range(6), 
            header=None, names=['chr', 'dataset', 'type', 'start','end', '.', 'strand', 'na', 'info'])

full_gencode = full_gencode[full_gencode['type']=='transcript']
full_gencode['transcript_id'] = full_gencode['info'].str.split(';').str[1].str.split('\"').str[-2]
full_gencode['transcript_type'] = full_gencode['info'].str.split(';').str[4].str.split('\"').str[-2]
full_gencode['gene_type'] = full_gencode['info'].str.split(';').str[2].str.split('\"').str[-2]
full_gencode['gene_name'] = full_gencode['info'].str.split(';').str[3].str.split('\"').str[-2]


# filter to just protein coding
protein_gencode = full_gencode[full_gencode['transcript_type']=='protein_coding']

# no duplicates?
len(protein_gencode[protein_gencode['transcript_id'].duplicated()]) == 0

True

In [5]:
# get the gene transcripts in each TAD
for tissue_id in tqdm(tissue_ids):
    # load in expression
    expression_df = pd.read_csv(f'{prefix}/{expression_dir}/{tissue_id}.v8.normalized_expression.bed', sep='\t')
    protein_gencode_filtered = protein_gencode[protein_gencode['transcript_id'].isin(expression_df['gene_id'])]

    # Iterate over each row of tad_df
    for tad_index, tad_row in tad_df.iterrows():
        # filter for if the genes are in the tad
        in_tad_filter = ((protein_gencode_filtered['chr'] == tad_row['chr']) & 
                    (protein_gencode_filtered['start'] >= tad_row['start']) & 
                    (protein_gencode_filtered['end'] <= tad_row['end']))
    

        # store list of transcripts, genes, and number of genes
        tad_df.loc[tad_index, 'Transcripts'] = ','.join(protein_gencode_filtered.loc[in_tad_filter]['transcript_id'])
        tad_df.loc[tad_index, 'Genes'] = ','.join(protein_gencode_filtered.loc[in_tad_filter]['gene_name'])
        tad_df.loc[tad_index, 'N_genes'] = int(sum(in_tad_filter))

    # add tissue
    tad_df['Tissue'] = tissue_id
    # only write out where there are genes in the tad
    tad_out_df = tad_df[~(tad_df['N_genes'] == 0)]
    tad_out_df[['N_genes', 'Transcripts', 'Genes', 'Chromosome', 'Tissue']].to_csv(f'/home/klawren/oak/pcqtls/data/processed/proteincoding/clusters_tad/{tissue_id}_clusters_all_chr.csv')

  0%|          | 0/13 [00:00<?, ?it/s]